In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
modelName = "NousResearch/Llama-2-7b-chat-hf" # The model we will fine tune -- from Hugging Face
dataset_name = "PiyushLavaniya/HTML_Dataset_for_LLama2_Finetuning"
newModel = "Llama2-FineTuned" # The name of the fine tuned model


#########
# QLoRA parameters
#########

lora_r = 64 # LoRA attention dimension (Rank)
lora_alpha = 16 # Alpha parameter for LoRA scaling
lora_dropout = 0.1 # Dropout probabilty for LoRA layers

#########
# BitsAndBytes parameters --- For Quantization
#########

use_4bit = True # Activate 4-bit precision base model loading
bnb_4bit_compute_dtype = "float16" # Compute datatype for 4-bit base models
bnb_4bit_quant_type = "nf4" # Quantization type (fp4 or nf4)
use_nested_quant = False # Do not activate nested quantization for 4-bit base models (double quantization)n mn

#########
# TrainingArguments parameters
#########

outputDirectory = "./results" # Directory where the model's predictions and checkpoints will be stored

num_train_epochs = 1 # Number of training epochs

fp16 = False
bf16 = False

per_device_train_batch_size = 4 # Batch size per GPU for training
per_device_eval_batch_size = 4 # Btach size per GPU for evaluation

gradient_accumulation_steps = 1 # Number of update steps to accumulate the gradients for
gradient_checkpointing = True # Enable gradient checkpointing
max_grad_norm = 0.3 # Maximum gradient normal (gradient clipping)

learning_rate = 2e-4 # Initial learning rate (AdamW optimizer)

weight_decay = 0.001 # Weight decay to apply to all layerjs except bias/LayerNorm weights

optim = "paged_adamw_32bit" # Optimizer to use

lr_scheduler_type = "cosine" # Learning rate schedule

max_steps = -1 # Number of training steps (overrides num_train_epochs)

warmup_ratio = 0.03 # Ratio of steps for linear warmup (from 0 to learning rate)

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

save_steps = 0 # Save checkpoint every X updates steps

logging_steps = 25 # Log every X updates steps

#########
# SFT parameters
#########

max_seq_length = None # Maximum sequence length to use

packing = False # Pack multiple short examples in the same input sequence to increase efficiency

device_map = {"": 0} # Load the entire model on the GPU 0

In [ ]:
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    modelName,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(modelName, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=outputDirectory,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

1691

In [ ]:
# Train model
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,1.969300
50,1.493700
75,0.935800
100,1.041900
125,0.855800
150,1.058000
175,0.912400
200,0.986900
225,0.924700
250,0.963500


TrainOutput(global_step=500, training_loss=1.0216966819763185, metrics={'train_runtime': 1025.911, 'train_samples_per_second': 1.949, 'train_steps_per_second': 0.487, 'total_flos': 5062364922839040.0, 'train_loss': 1.0216966819763185, 'epoch': 1.0})

In [ ]:
# Save trained model
trainer.model.save_pretrained(newModel)

In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "Create an HTML page to display a table of students. Data of students: First name, LastName and Age"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST] \n")
print(result[0]['generated_text'])

<s>[INST] Create an HTML page to display a table of students. Data of students: First name, LastName and Age [/INST] 
<html>
<head>
  <title>Student Table</title>
</head>
<body>
  <table>
    <tr>
      <th>First Name</th>
      <th>Last Name</th>
      <th>Age</th>
    </tr>
    <tr>
      <td>John</td>
      <td>Doe</td>
      <td>25</td>
    </tr>
    <tr>
      <td>Jane</td>
      <td>Smith</td>
      <td>30</td>
    </tr>
    <tr>
      <td>Bob</td>
      <td>Jones
